<a href="https://colab.research.google.com/github/avneesh-nolkha/EVA5/blob/master/Session13/EVA5_Assignment13_YoloV3_first50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/theschoolofai/YoloV3.git

Cloning into 'YoloV3'...
remote: Enumerating objects: 156, done.
remote: Total 156 (delta 0), reused 0 (delta 0), pack-reused 156
Receiving objects: 100% (156/156), 12.74 MiB | 36.05 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!ls

gdrive	sample_data  YoloV3


In [5]:
import time
import glob
import torch
import os

from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.6.0+cu101 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [6]:
cd YoloV3/

/content/YoloV3


In [7]:
!ls

 annotation_tool   output	  results.txt	     train.py
 cfg		   __pycache__	  test_batch0.png   'ubdivisions=1'
 data		   README.md	  test.py	     utils
 detect.py	   results.json   ting		     vim.exe.stackdump
 models.py	   results.png	  train_batch0.png


In [8]:
mkdir weights


In [9]:
cp ../gdrive/'My Drive'/eva5/ass13/yolov3-spp-ultralytics.pt weights/

In [ ]:
#!cp ../gdrive/'My Drive'/eva5/ass13/custom/weights/best.pt weights/best.pt
# cp ../gdrive/'My Drive'/eva5/ass13/custom/weights/last.pt weights/last.pt

In [10]:
!ls weights

yolov3-spp-ultralytics.pt


In [11]:
# Copy and add custom dataset

!cp ../gdrive/'My Drive'/eva5/ass13/YoloV3_Dataset.zip .
!unzip YoloV3_Dataset.zip
!ls YoloV3_Dataset

Streaming output truncated to the last 5000 lines.
  inflating: __MACOSX/YoloV3_Dataset/Labels/._Mimg_108.txt  
  inflating: YoloV3_Dataset/Labels/z037 .txt  
  inflating: __MACOSX/YoloV3_Dataset/Labels/._z037 .txt  
  inflating: YoloV3_Dataset/Labels/images (52).txt  
  inflating: __MACOSX/YoloV3_Dataset/Labels/._images (52).txt  
  inflating: YoloV3_Dataset/Labels/Bimg_5.txt  
  inflating: __MACOSX/YoloV3_Dataset/Labels/._Bimg_5.txt  
  inflating: YoloV3_Dataset/Labels/images (1).txt  
  inflating: __MACOSX/YoloV3_Dataset/Labels/._images (1).txt  
  inflating: YoloV3_Dataset/Labels/Himage_085.txt  
  inflating: __MACOSX/YoloV3_Dataset/Labels/._Himage_085.txt  
  inflating: YoloV3_Dataset/Labels/C27.txt  
  inflating: __MACOSX/YoloV3_Dataset/Labels/._C27.txt  
  inflating: YoloV3_Dataset/Labels/SImage_105.txt  
  inflating: __MACOSX/YoloV3_Dataset/Labels/._SImage_105.txt  
  inflating: YoloV3_Dataset/Labels/photo-of-smiling-bearded-construction-worker-in-green-vest-picture-id172688507

In [12]:
import os
import os.path
from PIL import Image

# Script to write sizes of images into .shapes file
def write_shapes(imgpaths, shapefile, append=""):
  with open(imgpaths) as f1, open(shapefile, "w") as f2:
    for line in f1:
      filepath = append + line.strip()
      if os.path.isfile(filepath):
        im = Image.open(filepath)
        width, height = im.size
        f2.write(str(width) + " " + str(height) + "\n")
      else:
        print("File does not exist, delete from paths:", filepath)
        #f2.write(str(0) + " " + str(0) + "\n")  
#write_shapes("YoloV3_Dataset/YoloV3_Dataset/train.txt", "train.shapes", "YoloV3_Dataset/YoloV3_Dataset/")


In [13]:
# Delete existing images and labels, and copy new ones from own gdrive folder
!rm data/customdata/images/*
!rm data/customdata/labels/*

!cp YoloV3_Dataset/Images/* data/customdata/images/
!cp YoloV3_Dataset/Labels/* data/customdata/labels/


# In custom cfg file, change classes=4 and filters=(4+1+4 (ie classes) ) * 3 as per
# https://github.com/theschoolofai/YoloV3
# Also change these params: for shorter run
  # burn_in to 100
  # max_batches to 5000
  # steps to 4000,4500
## Running with burn_in = 30 for 50 epoch run

In [14]:
# Copy custom config files
!cp ../gdrive/'My Drive'/eva5/ass13/custom/custom.names data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/custom.data data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/custom.txt data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/custom_train.txt data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/custom_test.txt data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/yolov3-custom.cfg cfg/yolov3-custom.cfg
!cp ../gdrive/'My Drive'/eva5/ass13/custom/yolov3-custom_100.cfg cfg/yolov3-custom_100.cfg
!cp ../gdrive/'My Drive'/eva5/ass13/custom/custom.shapes data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/train.shapes data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/test.shapes data/customdata/


In [16]:
!ls data/customdata

custom.data  custom.names  custom.shapes  custom.txt  images  labels


In [15]:
# These files have a problem, label coordinate > 1. Manually fix or else delete label:
# ./data/customdata/labels/8c7a3576.txt
# ./data/customdata/labels/8c7b2016.txt
# ./data/customdata/labels/8c7d69b6.txt

!rm data/customdata/labels/8c7a3576.txt
!rm data/customdata/labels/8c7b2016.txt
!rm data/customdata/labels/8c7d69b6.txt

# Image not found: remove from custom_train and custom_test.txt // done already
# File does not exist, delete from paths: YoloV3_Dataset/YoloV3_Dataset/Images/Aimgg_005.jpg
# File does not exist, delete from paths: YoloV3_Dataset/YoloV3_Dataset/Images/Images/Aimg_023.jpg
# File does not exist, delete from paths: YoloV3_Dataset/YoloV3_Dataset/images/img16.jpg
# File does not exist, delete from paths: YoloV3_Dataset/YoloV3_Dataset/images/img27.jpg
# File does not exist, delete from paths: YoloV3_Dataset/YoloV3_Dataset/images/img2.jpg
# File does not exist, delete from paths: YoloV3_Dataset/YoloV3_Dataset/images/img3.jpg
#
#File does not exist, delete from paths: YoloV3_Dataset/YoloV3_Dataset/Images/ImageYolo.jpg
# File does not exist, delete from paths: YoloV3_Dataset/YoloV3_Dataset/Images/M/img_010.jpg

# Save checkpoints in gdrive.. change in train.py

In [21]:
# Copy images from test video. framerate was set to 10 while extraction, use the same while creating video back
!cp ../gdrive/'My Drive'/eva5/ass13/testvid.zip .
!unzip testvid.zip


Archive:  testvid.zip
   creating: testvid/
  inflating: testvid/image-001.png   
  inflating: testvid/image-002.png   
  inflating: testvid/image-003.png   
  inflating: testvid/image-004.png   
  inflating: testvid/image-005.png   
  inflating: testvid/image-006.png   
  inflating: testvid/image-007.png   
  inflating: testvid/image-008.png   
  inflating: testvid/image-009.png   
  inflating: testvid/image-010.png   
  inflating: testvid/image-011.png   
  inflating: testvid/image-012.png   
  inflating: testvid/image-013.png   
  inflating: testvid/image-014.png   
  inflating: testvid/image-015.png   
  inflating: testvid/image-016.png   
  inflating: testvid/image-017.png   
  inflating: testvid/image-018.png   
  inflating: testvid/image-019.png   
  inflating: testvid/image-020.png   
  inflating: testvid/image-021.png   
  inflating: testvid/image-022.png   
  inflating: testvid/image-023.png   
  inflating: testvid/image-024.png   
  inflating: testvid/image-025.png   
  infl

In [17]:
# Training for 50 epochs and resuming with saved weights later. 
# Taking ~6 hrs and disconnects often..
# Make this change inside train.py to save the best weights into google drive folder in case this session disconnects.
"""
if (best_fitness == fi) and not final_epoch:
                torch.save(chkpt, best)
                torch.save(chkpt, '../gdrive/My Drive/eva5/ass13/custom/weights/best.pt')
"""
# Using these cfg params: for 50 epoch run
  # burn_in to 30
  # max_batches to 5000
  # steps to 4000,4500

!python train.py --data data/customdata/custom.data --batch 16 --cache --cfg cfg/yolov3-custom.cfg --epochs 50 


Namespace(accumulate=4, adam=False, batch_size=16, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=50, evolve=False, img_size=[512], multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/yolov3-spp-ultralytics.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

2020-10-26 09:17:50.131869: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Caching labels (3025 found, 134 missing, 38 empty, 0 duplicate, for 3197 images): 100% 3197/3197 [00:00<00:00, 5034.36it/s]
Caching images (1.8GB): 100% 3197/3197 [00:22<00:00, 140.33it/s]
Reading image shapes: 100% 317/317 [00:00<00:00, 3363.86it/s]
Caching labels (

## Redoing training for further epochs

In [22]:
# Test with the video  images
!python detect.py --conf-thres 0.1 --source 'testvid'  --output out_out 

Namespace(agnostic_nms=False, augment=False, cfg='cfg/yolov3-custom.cfg', classes=None, conf_thres=0.1, device='', fourcc='mp4v', half=False, img_size=512, iou_thres=0.6, names='data/customdata/custom.names', output='out_out', save_txt=False, source='testvid', view_img=False, weights='weights/last.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
image 1/779 testvid/image-001.png: 320x512 Done. (0.017s)
image 2/779 testvid/image-002.png: 320x512 1 hardhats, 1 vests, Done. (0.016s)
image 3/779 testvid/image-003.png: 320x512 1 hardhats, Done. (0.016s)
image 4/779 testvid/image-004.png: 320x512 1 hardhats, 1 vests, Done. (0.016s)
image 5/779 testvid/image-005.png: 320x512 1 hardhats, 1 vests, Done. (0.016s)
image 6/779 testvid/image-006.png: 320x512 1 hardhats, Done. (0.016s)
image 7/779 testvid/image-007.png: 320x512 1 hardhats, 1 vests, Done. (0.016s)
image 8/779 test

In [23]:
# Images were extracted with a framerate of 10, use the same one to stitch video back

!ffmpeg -framerate 10 -i out_out/image-%03d.png out_video.mp4


ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [24]:
!cp out_video.mp4 ../gdrive/'My Drive'/eva5/ass13/custom/out_video.mp4

In [ ]:
# Saving weights again at the end for backup
#!cp weights/best.pt ../gdrive/'My Drive'/eva5/ass13/custom/weights/best.pt
!cp weights/last.pt ../gdrive/'My Drive'/eva5/ass13/custom/weights/last.pt


In [ ]:
!cp weights/best.pt ../gdrive/'My Drive'/eva5/ass13/custom/weights/best.pt
